<style>.md-sidebar--secondary {display: none !important;}</style>

# FCFS with AP and Product Restrictions

In this example, on top of product restrictions we also add advance purchase (AP)
restrictions to the simulation.  These restrictions close lower priced fare class
products as the day of departure approaches.

In this example, we will also demostrate adding yaml configuration content in a 
Jupyter notebook before loading configs, and adding a snapshot filter to the simulation.

In [ ]:
import passengersim as pax
pax.versions()

In prior examples, the configuration were given in one or more external file, passed
to the `from_yaml` constructor as arguments.  We can also craft multi-line yaml content
directly in a Python string, and pass that directly to the same contructor.  Here,
we will turn the `disable_ap` switch off, and add a snapshot filter.

In [ ]:
with_AP = """
simulation_controls:
  disable_ap: false

snapshot_filters:
- type: leg_untruncation
  airline: AL1
  sample: 400
  flt_no: 101
  directory: snapshots/em
"""

Snapshot filters can be triggered at limited specific points within a large simulation,
and they will write out details intermediate results for that specific point.  The example
snapshot filter here will provide extra output into the leg untruncation for a single flight
on a single sample in the simulation.  Storing this level of detail for every flight for every
sample would generally be gratuitous, slowing the simulation significantly and writing out
excessively large files, but the snapshots give the ability to see and explore details from 
deep inside the simulation.

In [ ]:
cfg = pax.Config.from_yaml([
    "network/01-base.yaml", 
    "network/02-buyup.yaml", 
    with_AP
])

In [ ]:
cfg.simulation_controls.num_trials = 4

In [ ]:
sim = pax.Simulation(cfg)

In [ ]:
summary = sim.run()

In [ ]:
summary.fig_carrier_revenues()

In [ ]:
summary.fig_carrier_load_factors()

In [ ]:
summary.fig_fare_class_mix()

In [ ]:
summary.fig_bookings_by_timeframe()

In [ ]:
summary.to_xlsx("outputs/3mkt-03.xlsx")

[Download 3mkt-03.xlsx](./outputs/3mkt-03.xlsx)

## Comparing against Targets

In [ ]:
import targets

target = targets.load(3, cfg)

In [ ]:
from passengersim import contrast

comps = contrast.Contrast({
    "simulation": summary,
    "target": target,
})

In [ ]:
comps.fig_bookings_by_timeframe(by_carrier="AL1")

In [ ]:
comps.fig_carrier_revenues()

In [ ]:
comps.fig_fare_class_mix()

In [ ]:
comps.fig_bookings_by_timeframe(by_carrier="AL1", by_class=True)

In [ ]:
comps.fig_leg_forecasts(of=["mu", "sigma"], by_flt_no=101)

In [ ]:
comps.fig_leg_forecasts(of=["mu", "sigma"], by_flt_no=111)

In [ ]:
contrast.fig_leg_forecasts(comps, of=["mu", "sigma"], by_flt_no=101, agg_booking_classes=True)